In [16]:
%%javascript
var width = window.innerWidth || document.documentElement.clientWidth || document.body.clientWidth;
var height = window.innerHeight || document.documentElement.clientHeight || document.body.clientHeight;

IPython.notebook.kernel.execute("windowSize = (" + width + "," + height + ")");
// suitable for small screens
nbpresent.mode.tree.set(
    ["app", "theme-manager", "themes", "my-theme"], 
    {
    palette: {
        "blue": { id: "blue", rgb: [0, 153, 204] },
        "black": { id: "black", rgb: [0, 0, 0] },
        "white": { id: "white", rgb: [255, 255, 255] },
        "red": { id: "red", rgb: [240, 32, 32] },
        "gray": { id: "gray", rgb: [128, 128, 128] },
    },
    backgrounds: {
        "my-background": {
            "background-color": "white"
        }
    },
    "text-base": {
        "font-family": "Georgia",
        "font-size": 2.5
    },
    rules: {
        h1: {
            "font-size": 5.5,
            color: "blue",
            "text-align": "center"
        },
        h2: {
            "font-size": 3,
            color: "blue",
            "text-align": "center"
        },
        h3: {
            "font-size": 3,
            color: "black",
        },
        "ul li": {
            "font-size": 2.5,
            color: "black"
        },
        "ul li ul li": {
            "font-size": 2.0,
            color: "black"
        },
        "code": {
            "font-size": 1.6,
        },
        "pre": {
            "font-size": 1.6,
        }
    }
});

<IPython.core.display.Javascript object>

# Sequence Alignment

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/CityGrid.jpg" width="400" style="centerImg">

* Relating sequence alignment to our Manhattan Tour Problem
* Go over Midterm exam
* You should see grades for PS#1, PS#2, and Midterm online
* PS#3 is due tonight
* PS#4 will be posted before the weekend.
<p style="text-align: right; clear: right;">1</p>

# A Biological Dynamic Programming Problem

* How to measure the similarity between a pair of nucleotide or amino acid sequences
* When Motif-Searching we used Hamming distance as a measure of sequence similarity
* Is Hamming distance the best measure?
* How can we distinguish matches that occur by chance from slightly modified patterns?
* What sorts of modifications should we allow?

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/Clustal.gif" width="500">

<p style="text-align: right; clear: right;">2</p>

# Best Sequence Matches

* Depends on how you define *Best*
* Consider the two DNA sequences *v* and *w* :

<pre style="font-size: 160%;">
         v: TAGACAAT
         w: AGAGACAT
            11111111
</pre>


* The Hamming distance: *dH(v, w) = 8* is large but the sequences have similarity
* What if we allowed insertions and deletions?

<p style="text-align: right; clear: right;">3</p>

# Allowing Insertions and Deletions

* By shifting one sequence over one position:

<pre style="font-size: 160%;">
         v: _TAGACAAT
         w: AGAGACA_T
            110000010
</pre>

* The edit distance: *dH(v, w) = 3*.
* Hamming distance neglects insertions and deletions

<p style="text-align: right; clear: right;">4</p>

# Edit Distance

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/Levenshtein.jpg" style="float: right;">
* Levenshtein introduced the notion of an “edit distance” between two strings as the minimum number of elementary operations (insertions, deletions, and substitutions) to transform one string into the other in 1965.
<br><br>
* d(v,w) = Minimum number of elementary operations to transform *v &rarr; w*
<br><br>
* Computing Hamming distance is a trivial task
<br><br>
* Computing edit distance is less trivial
<br><br>
<p style="text-align: right; clear: right;">5</p>

# Edit Distance: Example

<pre style="font-size: 120%;">
    TGCATAT &rarr; ATCCGAT in 5 steps

    TGCATAT &rarr; (DELETE last T)
    TGCATA  &rarr; (DELETE last A)
    TGCAT   &rarr; (INSERT A at front)
    ATGCAT  &rarr; (SUBSTITUTE C for G)
    ATCCAT  &rarr; (INSERT G before last A) 
    ATCCGAT   (Done)
</pre>

What is the edit distance?  5?

<p style="text-align: right; clear: right;">6</p>

# Edit Distance: Example (2<sup>nd</sup> Try)

<pre style="font-size: 120%;">
    TGCATAT &rarr; ATCCGAT in 4 steps

    TGCATAT  &rarr; (INSERT A at front)
    ATGCATAT &rarr; (DELETE 2nd T)
    ATGCAAT  &rarr; (SUBSTITUTE G for 2nd A)
    ATGCGAT  &rarr; (SUBSTITUTE C for 1st G)
    ATCCGAT    (Done)
</pre>   

Is 4 the minimum edit distance?    3?

* A little jargon: Since the effect of insertion in one string can be accomplished via a deletion in the other string these two operations are correlated. Often algorithms will consider them together as a single operation called INDEL

<p style="text-align: right; clear: right;">7</p>

# Longest Common Subsequence

* A special case of edit distance where no *substitutions* are allowed
* A subsequence need not be contiguous, but the symbol order must be preserved<br>
	Ex. If v = ATTGCTA then AGCA and TTTA are subsequences of v, but TGTT and ACGA are not
* All substrings of *v* are subsequences, but not vice versa
* The edit distance, *d*, is related to the length of the LCS, *s*, by:

$$d(u,w) = len(v) + len(w) – 2 s(u,w)$$


<pre style="font-size: 120%;">
        ANUNCLEIKE
        UNCBEATDUKE
        
        anUNC_lE____iKE  10 - 6 = 4 
        __UNCb_Eatdu_KE  11 - 6 = 5
</pre>

<p style="text-align: right; clear: right;">8</p>

# LCS as a Dynamic Program

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/LCSgrid.png" width="300" class="centerImg">

* All possible possible alignments can be represented as a path from the string’s beginning (source) to its end (destination)
* Horizontal edges add gaps in v. Vertical edges add gaps in w. Diagonal edges are a match  
* Notice that we’ve only included valid diagonal edges appear in our graph

<p style="text-align: right; clear: right;">9</p>

# Various Alignments

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/LCSpath1.png" width="360" style="float: right;">
* Introduce coordinates for the grid
* All valid paths from the source to the destination represent *some* alignment
<pre style="font-size: 120%;">
                    0 1 2 2 3 4 5 6 7 7
                    v A T _ G T T A T _
                    w A T C G T _ A _ C
                    0 1 2 3 4 5 5 6 6 7  
</pre>
* Path: (0,0), (1,1), (2,2), (2,3), (3,4), (4,5), (5,5), (6,6), (7,6), (7,7)
<br><br><br><br><br><br>

<p style="text-align: right; clear: right;">10</p>

# Alternate Alignment

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/LCSpath2.png" width="360" style="float: right;">
* Introduce coordinates for the grid
* All valid paths from the source to the destination represent some alignment
<pre style="font-size: 120%;">
                    0 1 2 2 3 4 5 6 6 7
                    v A T _ G T T A _ T
                    w A T C G _ T A C _
                    0 1 2 3 4 4 5 6 7 7  
</pre>
* Path: (0,0), (1,1), (2,2), (2,3), (3,4), (4,4), (5,5), (6,6), (6,7), (7,7)
<br><br><br><br><br><br>

<p style="text-align: right; clear: right;">11</p>

# Even Bad Alignments

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/LCSpath3.png" width="360" style="float: right;">
* Introduce coordinates for the grid
* All valid paths from the source to the destination represent some alignment
<pre style="font-size: 120%;">
            0 0 0 0 0 0 1 2 3 4 5 6 7 7
            v _ _ _ _ _ A T G T T A T _
            w A T C G T A _ _ _ _ _ _ C
            0 1 2 3 4 5 6 6 6 6 6 6 6 7 
</pre>
* Path: (0,0), (0,1), (0,2), (0,3), (0,4), (0,5), (1,6), (2,6), (3,6), (4,6), (5,6), (6,6), (7,6), (7,7)
<br><br><br><br><br>

<p style="text-align: right; clear: right;">12</p>

# What makes a good alignment?

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/CartoonAlignments.jpg" width="360" style="float: right;">
* Using as many diagonal segments (matches) as possible. Why?
<br><br>
* The end of a good alignment from (j...k) begins with a good alignment from (i..j)
<br><br>
* Same as Manhattan Tourist problem, where ***sites*** are only on the diagonal streets!
<br><br>
* Set diagonal street weights = 1, and horizontal and vertical weights = 0
<br><br><br><br><br><br><br><br><br><br>

<p style="text-align: right; clear: right;">13</p>

# Alignment: Dynamic Program

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/LCSDP1.png" width="500" class="centerImg">

<p style="text-align: right; clear: right;">14</p>

# Step 1

Initialize 1st row and 1st column to all zeroes.

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/LCSDP2.png" width="290" style="margin-left: 360px;">

* Note intersections/vertices are rows in this matrix

<p style="text-align: right; clear: right;">15</p>

# Step 2

Evaluate recursion for next row and/or next column

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/LCSDP3.png" width="600" class="centerImg">

<p style="text-align: right; clear: right;">16</p>

# Step 3

Continue recursion for next row and/or next column

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/LCSDP4.png" width="600" class="centerImg">

<p style="text-align: right; clear: right;">17</p>

# Step 4

Then one more row and/or column

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/LCSDP5.png" width="600" class="centerImg">

<p style="text-align: right; clear: right;">18</p>

# Step 5

And so on... 

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/LCSDP6.png" width="600" class="centerImg">

<p style="text-align: right; clear: right;">19</p>

# Step 6

And so on... 

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/LCSDP7.png" width="600" class="centerImg">

<p style="text-align: right; clear: right;">20</p>

# Step 7

Getting closer

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/LCSDP8.png" width="600" class="centerImg">

<p style="text-align: right; clear: right;">21</p>

# Step 8

Until we reach the last row and column

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/LCSDP9.png" width="600" class="centerImg">

<p style="text-align: right; clear: right;">22</p>

# Finally

We reach the end, which corresponds to an LCS of length 5

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/LCSDP10.png" width="600" class="centerImg">

Our answer includes both an optimal score, and a path back to find the alignment

<p style="text-align: right; clear: right;">23</p>

# LCS Code

Let's see how well the code matches the approach we sketched out...

In [6]:
from numpy import *

def findLCS(v, w):
    score = zeros((len(v)+1,len(w)+1), dtype="int32")
    backt = zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in xrange(1,len(v)+1):
        for j in xrange(1,len(w)+1):
            # find best score at each vertex
            if (v[i-1] == w[j-1]):
                score[i,j], backt[i,j] = max((score[i-1,j-1]+1,3), (score[i-1,j],1), (score[i,j-1],2))
            else:
                score[i,j], backt[i,j] = max((score[i-1,j],1), (score[i,j-1],2))
    return score, backt

v = "ATGTTAT"
w = "ATCGTAC"
s, b = findLCS(v,w)
for i in xrange(len(s)):
    print "%10s %-20s    %12s %-20s" % ('' if i else 'score =', str(s[i]), '' if i else 'backtrack =', str(b[i]))

   score = [0 0 0 0 0 0 0 0]        backtrack = [0 0 0 0 0 0 0 0]   
           [0 1 1 1 1 1 1 1]                    [0 3 2 2 2 2 3 2]   
           [0 1 2 2 2 2 2 2]                    [0 1 3 2 2 3 2 2]   
           [0 1 2 2 3 3 3 3]                    [0 1 1 2 3 2 2 2]   
           [0 1 2 2 3 4 4 4]                    [0 1 3 2 1 3 2 2]   
           [0 1 2 2 3 4 4 4]                    [0 1 3 2 1 3 2 2]   
           [0 1 2 2 3 4 5 5]                    [0 3 1 2 1 1 3 2]   
           [0 1 2 2 3 4 5 5]                    [0 1 3 2 1 3 1 2]   



* The same score matrix that we found by hand
* *"backtrack"* keeps track of the arrows that we used

<p style="text-align: right; clear: right;">24</p>

# Backtracking

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/Backtrack.png" width="360" class="centerImg">

In our example we used arrows {&darr;, &rarr;, &searr;}, which were represented in our matrix as {1,2,3} respectively. This numbering is arbitrrary, except that it does break ties in our implementation (matches > *w* deletions > *w* insertions).

Next we need code that finds a path from the end of our strings to the beginning using our *arrow* matrix


<p style="text-align: right; clear: right;">25</p>

# Code to extract our answer

We can write a simple recursive routine to return along the path of arrows that led to our best score.

In [7]:
def LCS(b,v,i,j):
    if ((i == 0) and (j == 0)):
        return ''
    if (b[i,j] == 3):
        result = LCS(b,v,i-1,j-1)
        result = result + v[i-1]
        return result
    else:
        if (b[i,j] == 2):
            return LCS(b,v,i,j-1)
        else:
            return LCS(b,v,i-1,j)
        
print LCS(b,v,b.shape[0]-1,b.shape[1]-1)

ATGTA


* Technically correct, ATGTA is the LCS

       w = ATcGT_A_c
       v = AT_GTtAt_ 

* Notice that we only need one of *v* or *w* since both contain the LCS
* Perhaps we would like to get more than just the LCS; for example, the correpsonding alignment.

<p style="text-align: right; clear: right;">26</p>

# An alignment of *v* and *w*

In [8]:
def Alignment(b,v,w,i,j):
    if ((i == 0) and (j == 0)):
        return ['','']
    if (b[i,j] == 3):
        result = Alignment(b,v,w,i-1,j-1)
        result[0] += v[i-1]
        result[1] += w[j-1]
        return result
    else:
        if (b[i,j] == 2):
            result = Alignment(b,v,w,i,j-1)
            result[0] += "_"
            result[1] += w[j-1]
            return result
        else:
            result = Alignment(b,v,w,i-1,j)
            result[0] += v[i-1]
            result[1] += "_"
            return result

align = Alignment(b,v,w,b.shape[0]-1,b.shape[1]-1)
print "v =", align[0]
print "w =", align[1]

v = AT_GTTAT_
w = ATCG_TA_C



<p style="text-align: right; clear: right;">27</p>

# Alignment with a Scoring Matrix

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/ScoreMatrix1.png" width="250" style="float: right;">
* Rather *edit distance* one could use a table with costs for every symbol aligned to any other
* Scoring matrices allow alignments to consider biological constraints 
* Alignments can be thought of as two sequences that differ due to mutations.  
* Some types of mutations are more common, or have little effect on the protein’s function, therefore some mismatch penalties, &delta;(v<sub>i</sub>, w<sub>j</sub>), should be less harsh than others.
<br><br>

**Example:**
* Although R (arginine) and K (lysine) are different amino acids, they might still have a positive score.
* Why? They are both positively charged amino acids and hydrophillic implying such a substitution may not greatly change function of protein.

<p style="text-align: right; clear: right;">28</p>

# Functional Conservation

* Amino acid changes that tend to preserve the electro-chemical properties of the original residue
  - Polar to polar (aspartate &rarr; glutamate)
  - Nonpolar to nonpolar (alanine &rarr; valine)
  - Similarly behaving residues (leucine &rarr; isoleucine)

* Common Amino acid substitution matrices
  - PAM
  - BLOSUM

* DNA substitution matrices
  - DNA is less conserved than protein sequences
  - Less effective to compare coding regions at nucleotide level

<p style="text-align: right; clear: right;">29</p>

# PAM

* **P**oint **A**ccepted **M**utation (Dayhoff et al.)
* 1 PAM = PAM<sub>1</sub> = 1% average change of all amino acid positions
  - After 100 PAMs of evolution, not every residue will have changed
    * some residues may have mutated several times
    * some residues may have returned to their original state
    * some residues may not changed at all
* PAM<sub>x</sub> ~ (PAM<sub>1</sub>)<sup>x</sup>
* PAM<sub>1</sub> is a widely used scoring matrix for very sequences
* PAM<sub>250</sub> is a widely used scoring matrix for evolutionarily distant sequences

<pre>
                 Ala  Arg  Asn  Asp  Cys  Gln  Glu  Gly  His  Ile  Leu  Lys ...
                  A    R    N    D    C    Q    E    G    H    I    L    K  ...
        Ala A    13    6    9    9    5    8    9   12    6    8    6    7  ...
        Arg R     3   17    4    3    2    5    3    2    6    3    2    9
        Asn N     4    4    6    7    2    5    6    4    6    3    2    5
        Asp D     5    4    8   11    1    7   10    5    6    3    2    5
        Cys C     2    1    1    1   52    1    1    2    2    2    1    1
        Gln Q     3    5    5    6    1   10    7    3    7    2    3    5
        ...
        Trp W     0    2    0    0    0    0    0    0    1    0    1    0
        Tyr Y     1    1    2    1    3    1    1    1    3    2    2    1
        Val V     7    4    4    4    4    4    4    4    5    4   15   10
</pre>

<p style="text-align: right; clear: right;">30</p>

# BLOSUM

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/Blosum50.png" width="400" style="float: right; margin-left: 100px;">

* **Blo**ck **Su**bstitution **M**atrix 
* Scores derived from observations of the frequencies of substitutions in blocks of *local alignments* in related proteins
* Matrix name indicates evolutionary distance
* BLOSUM50 was created using actual sequences sharing no more than 50% identity

<p style="text-align: right; clear: right;">31</p>

# Global Alignment using a scoring matrix

In [45]:
import numpy

def GlobalAlign(v, w, scorematrix, indel):
    s = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    b = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in xrange(1,len(v)+1):
        for j in xrange(1,len(w)+1):
            score = s[i-1,j-1] + scorematrix[v[i-1],w[j-1]]
            vskip = s[i-1,j] + indel
            wskip = s[i,j-1] + indel
            s[i,j] = max(vskip, wskip, score)
            if (s[i,j] == vskip):
               b[i,j] = 1
            elif (s[i,j] == wskip):
               b[i,j] = 2
            else:
               b[i,j] = 3
    return (s, b)

match = {('A','A'):  2, ('A','C'): -1, ('A','G'):  0, ('A','T'): -1,
         ('C','A'): -1, ('C','C'):  2, ('C','G'): -1, ('C','T'):  0,
         ('G','A'):  0, ('G','C'): -1, ('G','G'):  2, ('G','T'): -1,
         ('T','A'): -1, ('T','C'):  0, ('T','G'): -1, ('T','T'):  2}

v = "TTCCGAGCGTTA"
w = "TTTCAGGTTA"

s, b = GlobalAlign(v,w,match,-1)
align = Alignment(b,v,w,b.shape[0]-1,b.shape[1]-1)
print "v =", align[0]
print "w =", align[1]

v = TTCCGAGCGTTA
w = TTTC_AG_GTTA


<p style="text-align: right; clear: right;">31</p>

# Local vs. Global Alignment

* The *Global Alignment Problem* tries to find the highest scoring path between vertices (0,0) and (n,m) in the edit graph.

* The *Local Alignment Problem* tries to find the highest scoring subpath between all vertex pairs $(i_1,j_1)$ and $(i_2, j_2)$ in the edit graph where $i_2 > i_1$ and $j_2 > j_1$.

* In an edit graph with negatively-weighted scores, a Local Alignment may score higher than a Global Alignment

Example:
* Global Alignment
<pre style="font-size: 80%;">
        --T—-CC-C-AGT—-TATGT-CAGGGGACACG—A-GCATGCAGA-GAC
          |  || |  ||  | | | |||    || | | |  | ||||   |
        AATTGCCGCC-GTCGT-T-TTCAG----CA-GTTATG—T-CAGAT--C
</pre>
* Local Alignment finds longer conserved segment
<pre style="font-size: 80%;">
                          tccCAGTTATGTCAGgggacacgagcatgcagagac
                             ||||||||||||
        aattgccgccgtcgttttcagCAGTTATGTCAGatc
</pre>

<p style="text-align: right; clear: right;">32</p>

# Local Alignments: Why?

* Two genes in different species may be similar over short conserved regions and dissimilar over remaining regions.
* Example:
  - Homeobox genes have a short region called the homeodomain that is highly conserved between species. 
  - A global alignment would not find the homeodomain because it would try to align the ENTIRE sequence


**Local Alignment Problem:**
* **Goal**: Find the best local alignment between two strings
* **Input**: Strings *v*, *w* and scoring matrix δ
* **Output**: Alignment of substrings of *v* and *w* whose alignment score is maximum among all possible alignment of all possible substrings

<p style="text-align: right; clear: right;">33</p>

# Local Alignment Approach

A local alignment is a subpath in a global alignment

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/LocalAlign.png" width="800" class="centerImg">

<p style="text-align: right; clear: right;">34</p>

# Brute Force Local Alignment

Find the best global alignment amoung all blocks $(i_1,j_1,i_2,j_2)$

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/LocalAlignBlocks.png" width="400" class="centerImg">

* Long run time $O(n^4)$:
  - In the grid of size n x n there are $O(n^2)$ vertices $(i_1,j_1)$ that may serve as a source. 
  - For each such vertex computing alignments from $(i_1,j_1)$ to $(i_2,j_2)$ takes $O(n^2)$ time.
  
* This can be remedied by giving free rides

<p style="text-align: right; clear: right;">35</p>

# Local Alignment: Free Rides

* ***Key Ideas:*** Add extra edges to our graph, consider all scores in matrix

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/FreeRides.png" width="500" class="centerImg">

* The dashed edges represent a *free ride* from (0,0) to any other node
* The largest value of s<sub>i,j</sub> over the *whole score matrix* is the end point of the best local alignment (instead of s<sub>n,m</sub>).

<p style="text-align: right; clear: right;">36</p>

# The Local Alignment Recurrence

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/LocalRecurrence.png" width="600" class="centerImg">

* The *zero* is our *free ride* that allows the node to restart with a score of 0 at any point
  - What does this imply?
* After solving for the entire score matrix, we then search for s<sub>i,j</sub> with the highest score, this is $(i_2,j_2)$
* We follow our back tracking matrix until we reach a *score* of 0, whose coordinate becomes $(i_1,j_1)$

<p style="text-align: right; clear: right;">37</p>

# Next Time

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/MindTheGap.png" width="360" class="centerImg">

* Alignment with Gap Penalities
* Multiple Alignment problem
* Can we do better than $O(MN)$?

<p style="text-align: right; clear: right;">38</p>